In [1]:
import flwr as fl
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from numpy import newaxis
import math
import os
import pandas as pd
import torch.nn as nn
from scipy.stats import chi2
from transformers import AutoTokenizer, AutoModelForMaskedLM, AutoModel
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Function
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from centralized import DomainAdaptationModel,ReviewDataset
from sklearn.model_selection import train_test_split

In [10]:
class DomainAdaptationClient(fl.client.NumPyClient):
    def __init__(self, model, train_loader, test_loader, device):
        self.model = model
        self.train_loader = train_loader
        self.test_loader = test_loader
        self.device = device

    # def get_parameters(self):
    #     return [val.cpu().numpy() for val in self.model.state_dict().values()]

    def set_parameters(self, parameters):
        state_dict = {k: torch.tensor(v) for k, v in zip(self.model.state_dict().keys(), parameters)}
        self.model.load_state_dict(state_dict, strict=True)

    def fit(self, parameters, config):
        self.set_parameters(parameters)
        print("Starting training...")
        self.model.train()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=0.001)
        for epoch in range(1):  # Perform a single epoch of training
            for batch_idx,batch in self.train_loader:
                input_ids, attention_mask, token_type_ids, labels = [x.to(self.device) for x in batch]
                sentiment_pred, domain_pred = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                loss = self.compute_loss(sentiment_pred, domain_pred, labels)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                  # Calculate loss
                total_loss += loss.item()
                # Calculate accuracy
                _, predicted = torch.max(sentiment_pred, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
                if batch_idx % 10 == 0:
                    print(f"Epoch [{epoch+1}/{1}], Batch [{batch_idx+1}/{len(self.train_loader)}], "
                          f"Loss: {loss.item():.4f}, Accuracy: {correct/total:.4f}")
         # Print final accuracy after epoch
        accuracy = correct / total
        print(f"Training complete. Accuracy: {accuracy:.4f}, Average Loss: {total_loss/len(self.train_loader):.4f}")
        return self.get_parameters(), len(self.train_loader.dataset), {}

    def compute_loss(self, sentiment_pred, domain_pred, labels):
        sentiment_loss = torch.nn.CrossEntropyLoss()(sentiment_pred, labels)
        domain_loss = torch.nn.CrossEntropyLoss()(domain_pred, labels)  # Assuming domain labels are in labels
        return sentiment_loss + domain_loss

    def evaluate(self, parameters, config):
        self.set_parameters(parameters)
        self.model.eval()
        correct = 0
        total = 0
        print("Starting evaluation...")
        with torch.no_grad():
            for batch in self.test_loader:
                input_ids, attention_mask, token_type_ids, labels = [x.to(self.device) for x in batch]
                sentiment_pred, domain_pred = self.model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
                _, predicted = torch.max(sentiment_pred, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        accuracy = correct / total
        return float(accuracy), len(self.test_loader.dataset),{"accuracy": accuracy}


In [3]:
torch.cuda.is_available()

False

In [11]:
#Model
model = DomainAdaptationModel()
tokenizer = AutoTokenizer.from_pretrained('jackaduma/SecBERT')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model.to(device)
# state_dict_path = 'C:/Users/hl100/Downloads/' + 'size250k_1epoch_1_model.bin'
# model.load_state_dict(torch.load(state_dict_path, map_location=device))
#Data
# df_full = pd.read_csv('D:\Hoc\SecBert\SecBERT\multilabel-train\dataset_capec.csv')
df_full = pd.read_csv('E:\Work_DatPT\Study\Master\SecBERT\dataset_capec.csv')
df_full['text'] = df_full['text'].str.replace('/',' ')
df_train = df_full.groupby('label').head(30)
# df_train = df_full
df_train = df_train.dropna(subset=['label'])
label_counts = df_train['label'].value_counts()
print(label_counts)
print(df_train.size)


train_df, test_df = train_test_split(df_train, test_size=0.2, random_state=42)
train_texts = train_df['text'].values
train_labels = train_df['label'].values
test_texts = test_df['text'].values
test_labels = test_df['label'].values

X_train, X_test, Y_train, Y_test = train_test_split(df_train['text'], df_train['label'],test_size=0.3, stratify=df_train['label'], shuffle = True)
df_train = pd.concat([X_train, Y_train], axis=1)
df_test = pd.concat([X_test, Y_test], axis=1)

# Tokenize the loaded texts for training and testing
train_dataset = ReviewDataset(df_train)
test_dataset = ReviewDataset(df_test)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False) 


<>:11: DeprecationWarning: invalid escape sequence '\W'


cpu
label
000 - Normal                              30
126 - Path Traversal                      30
66 - SQL Injection                        30
272 - Protocol Manipulation               30
310 - Scanning for Vulnerable Software    30
242 - Code Injection                      30
153 - Input Data Manipulation             30
194 - Fake the Source of Data             30
34 - HTTP Response Splitting              30
Name: count, dtype: int64
540


In [12]:
# Simulate clients
client = DomainAdaptationClient(model, train_loader, test_loader, device)
fl.client.start_numpy_client(server_address="localhost:8088", client=client)

	Instead, use `flwr.client.start_client()` by ensuring you first call the `.to_client()` method as shown below: 
	flwr.client.start_client(
		server_address='<IP>:<PORT>',
		client=FlowerClient().to_client(), # <-- where FlowerClient is of type flwr.client.NumPyClient object
	)
	Using `start_numpy_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: get_parameters message 5a5935b3-3494-4770-bbec-4dec3fcb67f7
INFO :      Sent reply
INFO :      Disconnect and shut down
